<div dir="rtl">

## למידת חיזוק במערכת פיזיקלית מתייצבת - אימון הסוכן

**שם התלמיד:** אייל פורת  
**שם המורה:** שי פרח  
**תאריך:** אפריל 2025

במחברת זו נאמן סוכן למידת חיזוק מסוג `DDPG (Deep Deterministic Policy Gradient)` באמצעות סביבת סימולצית הפיזיקה SimNet, אותה אימנו במחברת הייעודית.

`DDPG` הוא שיפור של ארכיטקטורת `Actor-Critic` הסטנדרטית, המתאים למרחבי פעולות רציפים כמו במצערת של הרובוט.


In [ ]:
# Uncomment the following lines to run in Google Colab

# %cd /content
# !git clone https://github.com/EyalPorat/ddpg-balancing-robot.git
# %cd ddpg-balancing-robot
# !git checkout simpler-state-3
# %cd /content/ddpg-balancing-robot/python/notebooks

# import sys
# sys.path.append('/content/ddpg-balancing-robot/python')  # Add the repo root to Python path

<div dir="rtl">

## ייבוא ספריות - Import Libraries

In [ ]:
import sys
sys.path.append('..')

import torch
import numpy as np
from pathlib import Path
import yaml
import matplotlib.pyplot as plt

from src.balancing_robot.models import Actor, Critic, ReplayBuffer, SimNet
from src.balancing_robot.environment import BalancerEnv
from src.balancing_robot.training import DDPGTrainer
from src.balancing_robot.visualization import plot_training_metrics, create_episode_animation

<div dir="rtl">

## טעינת קונפיגורציה - Configuration Loading

תחילה נטען את קבצי הקונפיגורציה המכילים את פרמטרי האימון והסביבה. באמצעותם ניתן בשלוט באופי האימון מבלי לצלול לתוך הקוד.
</div>

In [ ]:
# Load DDPG and environment configurations
with open('../configs/ddpg_config.yaml', 'r') as f:
    ddpg_config = yaml.safe_load(f)

with open('../configs/env_config.yaml', 'r') as f:
    env_config = yaml.safe_load(f)

with open('../configs/simnet_config.yaml', 'r') as f:
    simnet_config = yaml.safe_load(f)

# Create log directory
log_dir = Path('logs/ddpg_training')
log_dir.mkdir(parents=True, exist_ok=True)

<div dir="rtl">

## אתחול סביבת הסימולציה והמודלים - Initialize Environment and Models

נגדיר את הסוכן על שני המודלים שלו (Actor ו-Critic) ואת הסביבה SimNet נטען מהאחסון.
</div>

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Set random seeds from config
torch.manual_seed(ddpg_config["training"].get("random_seed", 42))
np.random.seed(ddpg_config["training"].get("random_seed", 42))

# Create environment
env = BalancerEnv(config_path="../configs/env_config.yaml", render_mode="rgb_array")

# Initialize SimNet from config
simnet = SimNet(
    state_dim=env.observation_space.shape[0],
    action_dim=env.action_space.shape[0],
    hidden_dims=simnet_config["model"]["hidden_dims"],
).to(device)

# Load the state dictionary
simnet.load_state_dict(torch.load("logs/simnet_training/simnet_final.pt", map_location=device)["state_dict"])
simnet.to(device)
simnet.eval()  # Set to evaluation mode

# Set the simnet in the environment
env.simnet = simnet

# Initialize trainer with config
trainer = DDPGTrainer(env=env, config_path="../configs/ddpg_config.yaml")

# Print model summaries
trainer.print_model_info()

<div dir="rtl">

## אימון - Training

תהליך האימון בלמידת חיזוק הוא תהליך איטרטיבי, בו הסוכן מוצב בסביבה ומנסה פעולות שונות בניסיון להגיע לערך החזר מירבי.

במקרה של DDPG - התיקונים של המודלים נעשים בשלבים, כאשר בתחילה מעדכנים את המבקר (Critic) באמצעות `TD-Error` (תיקון חלקי של ערך ההחזר הכולל באמצעות ערך החזר רגעי המתקבל מהסביבה) ולאחר מכן את השחקן (Actor) באמצעות `Policy Gradient` (ניסיון למקסם את ערך המבקר שקיבל הסוכן).

</div>



In [ ]:
# Extract training parameters from config
train_config = ddpg_config['training']

# Train agent
history = trainer.train(
    num_episodes=train_config['total_episodes'],
    max_steps=train_config['max_steps_per_episode'],
    batch_size=train_config['batch_size'],
    eval_freq=train_config['eval_frequency'],
    save_freq=train_config['save_frequency'],
    log_dir=log_dir
)

<div dir="rtl">

## ניתוח תוצאות האימון - Results Analysis

כמה נקודות חשובות:
- נביט על מדדי האימון לאורך זמן. השגיאה של המבקר (Critic) יורדת עם הזמן (כך שהמבקר מכיר טוב יותר את הסביבה ואת פונקציית ההחזר).
- ערך החיזוי של המבקר (Critic) גדל (כך שערך ההפסד של השחקן קטן).
- אורך הפרקים גדל עם הזמן, מה שמעיד על ייצוב מוצלח.
</div>

In [ ]:
# Plot training metrics
fig = plot_training_metrics(history, save_path=log_dir / 'training_metrics.png')
plt.show()

<div dir="rtl">

## רינדור של פרק בסימולציה - Episode Rendering

נביט על פרק מדומה, בו ניתן לראות את הרובוט מתייצב.

In [ ]:
# Create demonstration video
def collect_demo_episode(max_steps=500):
    state, _ = env.reset()
    states = []
    actions = []
    total_reward = 0

    for _ in range(max_steps):
        action = trainer.select_action(state, training=False)
        next_state, reward, done, truncated, info = env.step(action)
        states.append(state)
        actions.append(action)
        total_reward += reward
        
        if done:
            break
            
        state = next_state
    
    return np.array(states), np.array(actions), total_reward

# Collect several episodes and use the best one for visualization
num_episodes = 5
best_reward = float('-inf')
best_states = None
best_actions = None

for _ in range(num_episodes):
    states, actions, reward = collect_demo_episode()
    if reward > best_reward:
        best_reward = reward
        best_states = states
        best_actions = actions

print(f"Best episode reward: {best_reward:.2f}")

# Create and display animation
anim = create_episode_animation(
    states=best_states,
    actions=best_actions,
    save_path=log_dir / 'demo.mp4'
)
from IPython.display import HTML
HTML(anim.to_jshtml())

<div dir="rtl">

# שמירת המודל הסופי - Final Model Saving

</div>

In [ ]:
# Save final model with config and training history
torch.save({
    'actor_state_dict': trainer.actor.state_dict(),
    'critic_state_dict': trainer.critic.state_dict(),
    'training_history': history,
    'config': ddpg_config,
    'env_config': env_config,
    'metadata': {
        'state_dim': env.observation_space.shape[0],
        'action_dim': env.action_space.shape[0],
        'max_action': float(env.action_space.high[0]),
        'final_eval_reward': trainer.evaluate(num_episodes=10)
    }
}, log_dir / 'final_model.pt')

print("Training complete! Model saved with metadata.")